In [1]:
!pip install import_ipynb
!pip install ptflops
import import_ipynb
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image, make_grid
from ptflops import get_model_complexity_info
from utils import get_dataset, get_network, TensorDataset, get_time
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ptflops: filename=ptflops-0.6.9-py3-none-any.whl size=11712 sha256=efc265e0d377bf240b5a31300bb26078ae84a67aa93f73baa71bbd5eb5b27e64
  Stored in directory: /root/.cache/pip/wheels/b6/86/d5/cf62a3571b005f91cd9accefc5e10f40214538be997198afad
Successfully built ptflops
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from networks.ipynb


Hyperparameters

In [2]:
DATAPATH = './data'
SAVEPATH = './'
MODEL = 'ConvNet'
IMAGE_PER_CLASS = 10
EVAL_MODE = 'SS'
TRAIN_NET_EPOCH = 20
BATCH_SIZE = 256
LR_IMG = 1
LR_NET = 0.01
NUMBER_CLASSES = 10
ITERATIONS = 10
NUMBER_INITIALIZATIONS = 100
NUMBER_STEP_OPT = 50
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Task1 2.a

In [45]:
def evaluate(model,valid_data):
    model.eval()
    total = 0
    correct = 0
    for data in valid_data:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        output = model(inputs)
        output = torch.argmax(output, dim=1)
        correct += torch.sum(output == labels)
        total += BATCH_SIZE
    return correct / float(total)

In [46]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("CIFAR10", "./data")
model = get_network("ConvNet", channel, num_classes, im_size).to(DEVICE)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
validLoader = torch.utils.data.DataLoader(dst_test, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
val_acc_list, train_loss_list, epoch_list = [], [], []
cee = nn.CrossEntropyLoss()
print('%s training begins'%get_time())
for epoch in range(0, TRAIN_NET_EPOCH):
  model.train()
  train_loss = 0.

  for data in trainloader:
      inputs, labels = data
      inputs = inputs.to(DEVICE)
      labels = labels.to(DEVICE)
      optimizer.zero_grad()
      outputs = model(inputs)

      loss = cee(outputs, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

  valid_acc = evaluate(model, validLoader)
  print("Epoch: {} \tTraining Cost: {:.6f}\t "
      "Valid Acc: {}.".format(epoch, train_loss, valid_acc))
  val_acc_list.append(valid_acc)
  train_loss_list.append(train_loss)

print('%s training ends'%get_time())

macs, params = get_model_complexity_info(model, (3, im_size[0], im_size[1]), as_strings=True,
                                           print_per_layer_stat=False, verbose=True)
print("Valid Acc: {} \tFlops: {}".format(valid_acc, macs))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[2022-12-12 07:52:16] training begins
Epoch: 0 	Training Cost: 359.247167	 Valid Acc: 0.44941407442092896.
Epoch: 1 	Training Cost: 286.007982	 Valid Acc: 0.52392578125.
Epoch: 2 	Training Cost: 249.559107	 Valid Acc: 0.567089855670929.
Epoch: 3 	Training Cost: 226.537605	 Valid Acc: 0.57373046875.
Epoch: 4 	Training Cost: 210.482247	 Valid Acc: 0.610546886920929.
Epoch: 5 	Training Cost: 197.720117	 Valid Acc: 0.615527331829071.
Epoch: 6 	Training Cost: 187.176338	 Valid Acc: 0.623242199420929.
Epoch: 7 	Training Cost: 179.077719	 Valid Acc: 0.6280273795127869.
Epoch: 8 	Training Cost: 170.550790	 Valid Acc: 0.652148425579071.
Epoch: 9 	Training Cost: 163.824359	 Valid Acc: 0.6416015625.
Epoch: 10 	Training Cost: 157.849175	 Valid Acc: 0.6825195550918579.
Epoch: 11 	Training Cost: 152.773350	 Valid Acc: 0.683789074420929.
Epoch: 12 	Training Cost: 147.825725	 Valid Acc: 0.6783203482627869.
Epoch: 

In [47]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("MNIST", "./data")
model = get_network("ConvNet", channel, num_classes, im_size).to(DEVICE)
trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
validLoader = torch.utils.data.DataLoader(dst_test, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
val_acc_list_MNIST_ConvNet_real, train_loss_list_MNIST_ConvNet_real, epoch_list = [], [], []
cee = nn.CrossEntropyLoss()
print('%s training begins'%get_time())
for epoch in range(0, TRAIN_NET_EPOCH):
  model.train()
  train_loss = 0.

  for data in trainloader:
      inputs, labels = data
      inputs = inputs.to(DEVICE)
      labels = labels.to(DEVICE)
      optimizer.zero_grad()
      outputs = model(inputs)

      loss = cee(outputs, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

  valid_acc = evaluate(model, validLoader)
  print("Epoch: {} \tTraining Cost: {:.6f}\t "
      "Valid Acc: {}.".format(epoch, train_loss, valid_acc))
  val_acc_list_MNIST_ConvNet_real.append(valid_acc)
  train_loss_list_MNIST_ConvNet_real.append(train_loss)

print('%s training ends'%get_time())

macs, params = get_model_complexity_info(model, (channel, im_size[0], im_size[1]), as_strings=True,
                                           print_per_layer_stat=False, verbose=True)
print("Valid Acc: {} \tFlops: {}".format(valid_acc, macs))
print(val_acc_list_MNIST_ConvNet_real,train_loss_list_MNIST_ConvNet_real)


[2022-12-12 07:58:22] training begins
Epoch: 0 	Training Cost: 130.068001	 Valid Acc: 0.941699206829071.
Epoch: 1 	Training Cost: 35.584123	 Valid Acc: 0.9500976800918579.
Epoch: 2 	Training Cost: 25.099387	 Valid Acc: 0.95556640625.
Epoch: 3 	Training Cost: 20.181787	 Valid Acc: 0.9585937857627869.
Epoch: 4 	Training Cost: 17.249245	 Valid Acc: 0.9593750238418579.
Epoch: 5 	Training Cost: 15.276834	 Valid Acc: 0.9627929925918579.
Epoch: 6 	Training Cost: 13.692698	 Valid Acc: 0.964160144329071.
Epoch: 7 	Training Cost: 12.566437	 Valid Acc: 0.9639648795127869.
Epoch: 8 	Training Cost: 11.712995	 Valid Acc: 0.9652343988418579.
Epoch: 9 	Training Cost: 10.818626	 Valid Acc: 0.966601550579071.
Epoch: 10 	Training Cost: 10.223773	 Valid Acc: 0.9658203125.
Epoch: 11 	Training Cost: 9.717043	 Valid Acc: 0.967578113079071.
Epoch: 12 	Training Cost: 9.250067	 Valid Acc: 0.96630859375.
Epoch: 13 	Training Cost: 8.788854	 Valid Acc: 0.9676758050918579.
Epoch: 14 	Training Cost: 8.320960	 Valid 

Task1 2.b c

In [48]:
def evaluate(model,valid_data):
    model.eval()
    total = 0
    correct = 0
    for data in valid_data:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        output = model(inputs)
        output = torch.argmax(output, dim=1)
        correct += torch.sum(output == labels)
        total += BATCH_SIZE
    return correct / float(total)

def train_and_eval(model, image_syn_eval, syn_lable, dst_test):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  validLoader = torch.utils.data.DataLoader(dst_test, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss()
  valid_accs = []
  for epoch in range(TRAIN_NET_EPOCH):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    valid_acc = evaluate(model, validLoader)
    valid_accs.append(valid_acc)
    print("Epoch: {} \tTraining Cost: {:.6f}\t "
        "Valid Acc: {}.".format(epoch, train_loss, valid_acc))
  return valid_accs

def train_model(model, image_syn_eval, syn_lable, epochs):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss().to(DEVICE)
  for e in range(epochs):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()

def match_mse_loss(gw_syn, gw_real):
  gw_real_vec = []
  gw_syn_vec = []
  for ig in range(len(gw_real)):
      gw_real_vec.append(gw_real[ig].reshape((-1)))
      gw_syn_vec.append(gw_syn[ig].reshape((-1)))
  gw_real_vec = torch.cat(gw_real_vec, dim=0)
  gw_syn_vec = torch.cat(gw_syn_vec, dim=0)
  dis = torch.sum((gw_syn_vec - gw_real_vec) ** 2)
  return dis


def condensed_images_and_evaluate(dataset_name, model_name, eval_mode_name, randmon_init=False):
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(dataset_name, DATAPATH)
  class_index = {}
  data_save = []
  for i in range(len(dst_train)):
    lable = dst_train[i][1]
    if lable in class_index:
      class_index[lable].append(i)
    else:
      class_index[lable] = [i]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  images_all = torch.cat(images_all, dim=0).to(DEVICE)
  syn_image = []
  for c in range(num_classes):
    idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
    syn_image.append(images_all[idx_shuffle])
  syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
  syn_image.requires_grad = True
  syn_lable = []
  for c in range(num_classes):
    syn_lable += [c for i in range(IMAGE_PER_CLASS)]
  if randmon_init:
    syn_image = torch.randn(size=(num_classes*IMAGE_PER_CLASS, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=DEVICE)
  syn_lable = torch.tensor(syn_lable,device=DEVICE)
  optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
  criterion = nn.CrossEntropyLoss().to(DEVICE)
  total_loss = 0 
  print('%s training begins'%get_time())
  for exp in range(NUMBER_INITIALIZATIONS):
    model = get_network(model_name, channel, num_classes, im_size).to(DEVICE)
    model.train()
    net_parameters = list(model.parameters())

    for it in range(ITERATIONS):
      loss = torch.tensor(0.0).to(DEVICE)
      for c in range(num_classes):
        strat_index = c*IMAGE_PER_CLASS
        img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
        lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
        img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
        output_real = model(img_real)
        loss_real = criterion(output_real, lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_syn = model(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)
        loss += match_mse_loss(gw_syn, gw_real)

      total_loss += loss
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()

      image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
      train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)
      
  random = "Gaussian noise" if randmon_init else "real training images"
  save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%(model_name, dataset_name, random, IMAGE_PER_CLASS))
  image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
  save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
  make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
  net_eval = get_network(eval_mode_name, channel, num_classes, im_size).to(DEVICE)
  image_syn_eval = copy.deepcopy(syn_image.detach())

  print('%s training begins'%get_time())
  acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
  print('%s training end'%get_time())
  return acc_test

Condense MINIST with ConvNet

In [49]:
acc_list = condensed_images_and_evaluate("MNIST", "ConvNet", "ConvNet")
print(acc_list)

[2022-12-12 08:06:36] training begins
[2022-12-12 08:22:42] training begins
Epoch: 0 	Training Cost: 2.351295	 Valid Acc: 0.18271484971046448.
Epoch: 1 	Training Cost: 2.254751	 Valid Acc: 0.35009765625.
Epoch: 2 	Training Cost: 2.188896	 Valid Acc: 0.4886718690395355.
Epoch: 3 	Training Cost: 2.130546	 Valid Acc: 0.5750976800918579.
Epoch: 4 	Training Cost: 2.076948	 Valid Acc: 0.6249023675918579.
Epoch: 5 	Training Cost: 2.026360	 Valid Acc: 0.6583008170127869.
Epoch: 6 	Training Cost: 1.979302	 Valid Acc: 0.680859386920929.
Epoch: 7 	Training Cost: 1.935225	 Valid Acc: 0.6963867545127869.
Epoch: 8 	Training Cost: 1.894252	 Valid Acc: 0.7093750238418579.
Epoch: 9 	Training Cost: 1.855789	 Valid Acc: 0.719433605670929.
Epoch: 10 	Training Cost: 1.819694	 Valid Acc: 0.725878894329071.
Epoch: 11 	Training Cost: 1.785347	 Valid Acc: 0.732421875.
Epoch: 12 	Training Cost: 1.752800	 Valid Acc: 0.73828125.
Epoch: 13 	Training Cost: 1.721762	 Valid Acc: 0.7413086295127869.
Epoch: 14 	Trainin


Condense CIFAR10 with ConvNet

In [50]:
acc_list = condensed_images_and_evaluate("CIFAR10", "ConvNet", "ConvNet")
print(acc_list)

Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 08:25:02] training begins
[2022-12-12 08:41:22] training begins
Epoch: 0 	Training Cost: 2.320782	 Valid Acc: 0.11757812649011612.
Epoch: 1 	Training Cost: 2.281565	 Valid Acc: 0.12724609673023224.
Epoch: 2 	Training Cost: 2.255727	 Valid Acc: 0.12900391221046448.
Epoch: 3 	Training Cost: 2.234272	 Valid Acc: 0.13125000894069672.
Epoch: 4 	Training Cost: 2.214694	 Valid Acc: 0.13457031548023224.
Epoch: 5 	Training Cost: 2.196168	 Valid Acc: 0.13750000298023224.
Epoch: 6 	Training Cost: 2.178458	 Valid Acc: 0.13828125596046448.
Epoch: 7 	Training Cost: 2.161406	 Valid Acc: 0.1396484375.
Epoch: 8 	Training Cost: 2.145015	 Valid Acc: 0.14199219644069672.
Epoch: 9 	Training Cost: 2.129116	 Valid Acc: 0.14365234971046448.
Epoch: 10 	Training Cost: 2.113585	 Valid Acc: 0.14541016519069672.
Epoch: 11 	Training Cost: 2.098385	 Valid Acc: 0.14824219048023224.
Epoch: 12 	Training Cost: 2.083453	 Valid Acc: 0.

Task2 d

condense MINIST with ConvNet initialized with Gaussian noise

In [51]:
acc_list = condensed_images_and_evaluate("MNIST", "ConvNet", "ConvNet", True)
print(acc_list)

[2022-12-12 08:44:05] training begins
[2022-12-12 09:00:09] training begins
Epoch: 0 	Training Cost: 2.317122	 Valid Acc: 0.27783203125.
Epoch: 1 	Training Cost: 2.212241	 Valid Acc: 0.41484376788139343.
Epoch: 2 	Training Cost: 2.147240	 Valid Acc: 0.5035156607627869.
Epoch: 3 	Training Cost: 2.089736	 Valid Acc: 0.5556640625.
Epoch: 4 	Training Cost: 2.037832	 Valid Acc: 0.59423828125.
Epoch: 5 	Training Cost: 1.989833	 Valid Acc: 0.621386706829071.
Epoch: 6 	Training Cost: 1.945165	 Valid Acc: 0.6470703482627869.
Epoch: 7 	Training Cost: 1.903703	 Valid Acc: 0.6670898795127869.
Epoch: 8 	Training Cost: 1.864957	 Valid Acc: 0.6836914420127869.
Epoch: 9 	Training Cost: 1.828184	 Valid Acc: 0.696484386920929.
Epoch: 10 	Training Cost: 1.792968	 Valid Acc: 0.705078125.
Epoch: 11 	Training Cost: 1.759248	 Valid Acc: 0.714648425579071.
Epoch: 12 	Training Cost: 1.727322	 Valid Acc: 0.72216796875.
Epoch: 13 	Training Cost: 1.696894	 Valid Acc: 0.7295898795127869.
Epoch: 14 	Training Cost: 

Condense CIFAR10 with ConvNet initialized with Gaussian noise

In [52]:
acc_list = condensed_images_and_evaluate("CIFAR10", "ConvNet", "ConvNet", True)
print(acc_list)

Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 09:01:36] training begins
[2022-12-12 09:17:57] training begins
Epoch: 0 	Training Cost: 2.340953	 Valid Acc: 0.10654296725988388.
Epoch: 1 	Training Cost: 2.293925	 Valid Acc: 0.11083984375.
Epoch: 2 	Training Cost: 2.264696	 Valid Acc: 0.11376953125.
Epoch: 3 	Training Cost: 2.240717	 Valid Acc: 0.11572265625.
Epoch: 4 	Training Cost: 2.218818	 Valid Acc: 0.11884766072034836.
Epoch: 5 	Training Cost: 2.198152	 Valid Acc: 0.12216796725988388.
Epoch: 6 	Training Cost: 2.178453	 Valid Acc: 0.12373047322034836.
Epoch: 7 	Training Cost: 2.159516	 Valid Acc: 0.12763671576976776.
Epoch: 8 	Training Cost: 2.141193	 Valid Acc: 0.13027344644069672.
Epoch: 9 	Training Cost: 2.123413	 Valid Acc: 0.13134765625.
Epoch: 10 	Training Cost: 2.106058	 Valid Acc: 0.13320313394069672.
Epoch: 11 	Training Cost: 2.089163	 Valid Acc: 0.13496093451976776.
Epoch: 12 	Training Cost: 2.072722	 Valid Acc: 0.13720703125.
Epoc

Task1 3. Cross-architecture Generalization

In [ ]:
condensed_images_and_evaluate("CIFAR10", "ConvNet", "AlexNet")

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0 	Training Cost: 2.303045	 Valid Acc: 0.103515625.
Epoch: 1 	Training Cost: 2.302770	 Valid Acc: 0.10498046875.
Epoch: 2 	Training Cost: 2.302499	 Valid Acc: 0.10771484673023224.
Epoch: 3 	Training Cost: 2.302229	 Valid Acc: 0.111328125.
Epoch: 4 	Training Cost: 2.301961	 Valid Acc: 0.11494141072034836.
Epoch: 5 	Training Cost: 2.301696	 Valid Acc: 0.11572265625.
Epoch: 6 	Training Cost: 2.301436	 Valid Acc: 0.11884766072034836.
Epoch: 7 	Training Cost: 2.301175	 Valid Acc: 0.12236328423023224.
Epoch: 8 	Training Cost: 2.300918	 Valid Acc: 0.12421875447034836.
Epoch: 9 	Training Cost: 2.300663	 Valid Acc: 0.12451171875.
Epoch: 10 	Training Cost: 2.300409	 Valid Acc: 0.12636719644069672.
Epoch: 11 	Training Cost: 2.300155	 Valid Acc: 0.12705078721046448.
Epoch: 12 	Training Cost: 2.299904	 Valid Acc: 0.12773437798023224.
Epoch: 13 	Training Cost: 2.299656	 Valid Acc: 0.12666015326976776.
Epoch: 14 	Train

In [ ]:
condensed_images_and_evaluate("MNIST", "ConvNet", "AlexNet")

Epoch: 0 	Training Cost: 2.302315	 Valid Acc: 0.10664062947034836.
Epoch: 1 	Training Cost: 2.302051	 Valid Acc: 0.10810547322034836.
Epoch: 2 	Training Cost: 2.301785	 Valid Acc: 0.10986328125.
Epoch: 3 	Training Cost: 2.301523	 Valid Acc: 0.1123046875.
Epoch: 4 	Training Cost: 2.301263	 Valid Acc: 0.11416015774011612.
Epoch: 5 	Training Cost: 2.301003	 Valid Acc: 0.11591797322034836.
Epoch: 6 	Training Cost: 2.300747	 Valid Acc: 0.11874999850988388.
Epoch: 7 	Training Cost: 2.300490	 Valid Acc: 0.12050781399011612.
Epoch: 8 	Training Cost: 2.300235	 Valid Acc: 0.12216796725988388.
Epoch: 9 	Training Cost: 2.299982	 Valid Acc: 0.1240234375.
Epoch: 10 	Training Cost: 2.299733	 Valid Acc: 0.12607422471046448.
Epoch: 11 	Training Cost: 2.299485	 Valid Acc: 0.12802734971046448.
Epoch: 12 	Training Cost: 2.299238	 Valid Acc: 0.130859375.
Epoch: 13 	Training Cost: 2.298994	 Valid Acc: 0.1328125.
Epoch: 14 	Training Cost: 2.298749	 Valid Acc: 0.13544921576976776.
Epoch: 15 	Training Cost: 2.

Task1 4 application

In [ ]:
!pip install opacus==0.15.0
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
from opacus.privacy_engine import PrivacyEngine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 14.4 MB/s 


In [ ]:
def condensed_images_and_evaluate_priv(dataset_name, model_name, eval_mode_name):
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(dataset_name, DATAPATH)
  class_index = {}
  data_save = []
  for i in range(len(dst_train)):
    lable = dst_train[i][1]
    if lable in class_index:
      class_index[lable].append(i)
    else:
      class_index[lable] = [i]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  images_all = torch.cat(images_all, dim=0).to(DEVICE)
  syn_image = []
  for c in range(num_classes):
    idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
    syn_image.append(images_all[idx_shuffle])
  syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
  syn_image.requires_grad = True
  syn_lable = []
  for c in range(num_classes):
    syn_lable += [c for i in range(IMAGE_PER_CLASS)]
  syn_lable = torch.tensor(syn_lable,device=DEVICE)
  optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
  criterion = nn.CrossEntropyLoss().to(DEVICE)
  total_loss = 0 
  print('%s training begins'%get_time())
  for exp in range(NUMBER_INITIALIZATIONS):
    model = get_network(model_name, channel, num_classes, im_size).to(DEVICE)
    model.train()
    model_shadow = copy.deepcopy(model)
    model_shadow.train()
    model_parameters = list(model.parameters())
    model_shadow_parameters = list(model_shadow.parameters())
    optimizer_net_grad = torch.optim.SGD(model_shadow.parameters(), lr=LR_NET)  # optimizer for obtaining DP real gradient
    optimizer_net = torch.optim.SGD(model.parameters(), lr=LR_NET)  # optimizer_net for update model
    privacy_engine = PrivacyEngine(model_shadow, sample_size=len(dst_train), batch_size=BATCH_SIZE, alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)), noise_multiplier=1, max_grad_norm=0.1 )
    privacy_engine.attach(optimizer_net_grad)
    for it in range(ITERATIONS):
      loss = torch.tensor(0.0).to(DEVICE)
      for c in range(num_classes):
        strat_index = c*IMAGE_PER_CLASS
        img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
        lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
        img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
        model.train()
        model_shadow.load_state_dict(model.state_dict()) 
        model_shadow.train()
        model_shadow.zero_grad()
        output_real = model_shadow(img_real)
        loss_real = criterion(output_real, lab_real)
        loss_real.backward()
        optimizer_net_grad.step()  
        gw_real = list((p.grad.detach().clone() for p in model_shadow_parameters))

        output_syn = model(img_syn)
        loss_syn = criterion(output_syn, lab_syn)
        gw_syn = torch.autograd.grad(loss_syn, model_parameters, create_graph=True)
        loss += match_mse_loss(gw_syn, gw_real)

      total_loss += loss
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()
      image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
      train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)
      
  random = "Gaussian noise" if randmon_init else "real training images"
  save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%(model_name, dataset_name, random, IMAGE_PER_CLASS))
  image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
  save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
  make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
  net_eval = get_network(eval_mode_name, channel, num_classes, im_size).to(DEVICE)
  image_syn_eval = copy.deepcopy(syn_image.detach())

  print('%s training begins'%get_time())
  acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
  print('%s training end'%get_time())
  return acc_test

In [ ]:
condensed_images_and_evaluate_priv("MNIST", "ConvNet", "ConvNet")

/usr/local/lib/python3.8/dist-packages/opacus/privacy_engine.py:752: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/opacus/privacy_engine.py:236: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1113: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.8/dist-

Epoch: 0 	Training Cost: 11.076957	 Valid Acc: 0.45909997820854187.
Epoch: 1 	Training Cost: 8.729900	 Valid Acc: 0.524899959564209.
Epoch: 2 	Training Cost: 6.572433	 Valid Acc: 0.5862999558448792.
Epoch: 3 	Training Cost: 4.807154	 Valid Acc: 0.7067999839782715.
Epoch: 4 	Training Cost: 3.537501	 Valid Acc: 0.6723999977111816.
Epoch: 5 	Training Cost: 2.377632	 Valid Acc: 0.8036999702453613.
Epoch: 6 	Training Cost: 1.694393	 Valid Acc: 0.7843999862670898.
Epoch: 7 	Training Cost: 1.258266	 Valid Acc: 0.7897999882698059.
Epoch: 8 	Training Cost: 1.003713	 Valid Acc: 0.8093000054359436.
Epoch: 9 	Training Cost: 0.825141	 Valid Acc: 0.8032000064849854.
Epoch: 10 	Training Cost: 0.677408	 Valid Acc: 0.7958999872207642.
Epoch: 11 	Training Cost: 0.586079	 Valid Acc: 0.8047999739646912.
Epoch: 12 	Training Cost: 0.508415	 Valid Acc: 0.8024999499320984.
Epoch: 13 	Training Cost: 0.442412	 Valid Acc: 0.8050999641418457.
Epoch: 14 	Training Cost: 0.398143	 Valid Acc: 0.8025999665260315.
Epoc

task1 5 MHIST

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow.compat.v2 as tf
def data_loader(image_dir):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        image_dir+"train",
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        seed=123,
        image_size=(224, 224),
        batch_size=1)
    test_ds = tf.keras.utils.image_dataset_from_directory(
        image_dir+"test",
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        seed=123,
        image_size=(224, 224),
        batch_size=1)
    return train_ds, test_ds

In [5]:
train_ds, test_ds = data_loader("/content/drive/MyDrive/ece1512/images_with_label_majority/")

Found 2175 files belonging to 2 classes.
Found 977 files belonging to 2 classes.


In [6]:
def convert_ft_to_torch(train_ds):
  images = []
  lables = []
  count = 0
  for i in train_ds:

    count +=1
    img = i[0].numpy()
    c1 = img[0,:,:,0]
    c2 = img[0,:,:,1]
    c3 = img[0,:,:,2]
    images.append(np.array([c1, c2, c3]))
    lables.append(i[1].numpy().argmax())
  images = torch.tensor(images,device=DEVICE)
  lables = torch.tensor(lables,device=DEVICE)
  dst_train = TensorDataset(images, lables)
  return dst_train

In [7]:
train_ds = convert_ft_to_torch(train_ds)
test_ds = convert_ft_to_torch(test_ds)

<ipython-input-6-fabcd2647853>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  images = torch.tensor(images,device=DEVICE)


In [8]:
def evaluate(model,valid_data):
    model.eval()
    total = 0
    correct = 0
    for data in valid_data:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        output = model(inputs)
        output = torch.argmax(output, dim=1)
        correct += torch.sum(output == labels)
        total += BATCH_SIZE
    return correct / float(total)

def train_and_eval(model, image_syn_eval, syn_lable, dst_test):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  validLoader = torch.utils.data.DataLoader(dst_test, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss()
  valid_acc = 0
  for epoch in range(TRAIN_NET_EPOCH):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    valid_acc = evaluate(model, validLoader)
    print("Epoch: {} \tTraining Cost: {:.6f}\t "
        "Valid Acc: {}.".format(epoch, train_loss, valid_acc))
  return valid_acc

def train_model(model, image_syn_eval, syn_lable, epochs):
  dst_train = TensorDataset(image_syn_eval, syn_lable)
  trainloader = torch.utils.data.DataLoader(dst_train, batch_size=BATCH_SIZE, shuffle=True)
  optimizer = torch.optim.SGD(model.parameters(), lr=LR_NET)
  cee = nn.CrossEntropyLoss().to(DEVICE)
  for e in range(epochs):
    model.train()
    train_loss = 0.

    for data in trainloader:
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = cee(outputs, labels)
        loss.backward()
        optimizer.step()

def match_mse_loss(gw_syn, gw_real):
  gw_real_vec = []
  gw_syn_vec = []
  for ig in range(len(gw_real)):
      gw_real_vec.append(gw_real[ig].reshape((-1)))
      gw_syn_vec.append(gw_syn[ig].reshape((-1)))
  gw_real_vec = torch.cat(gw_real_vec, dim=0)
  gw_syn_vec = torch.cat(gw_syn_vec, dim=0)
  dis = torch.sum((gw_syn_vec - gw_real_vec) ** 2)
  return dis

In [ ]:
im_size= (224, 224)
channel = 3
num_classes = 2
dst_train = train_ds
dst_test = test_ds
randmon_init = False
class_index = {}
data_save = []
BATCH_SIZE = 4
for i in range(len(dst_train)):
  lable = int(dst_train[i][1])
  if lable in class_index:
    class_index[lable].append(i)
  else:
    class_index[lable] = [i]
images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
images_all = torch.cat(images_all, dim=0).to(DEVICE)

syn_image = []
for c in range(num_classes):
  idx_shuffle = np.random.permutation(class_index[c])[:IMAGE_PER_CLASS]
  syn_image.append(images_all[idx_shuffle])
syn_image = torch.cat(syn_image, dim=0).to(DEVICE)
syn_image.requires_grad = True

syn_image = torch.randn(size=(num_classes*IMAGE_PER_CLASS, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=DEVICE)
syn_lable = []
for c in range(num_classes):
  syn_lable += [c for i in range(IMAGE_PER_CLASS)]
syn_lable = torch.tensor(syn_lable,device=DEVICE)

optimizer_img = torch.optim.SGD([syn_image, ], lr=LR_IMG)
criterion = nn.CrossEntropyLoss().to(DEVICE)
total_loss = 0 
print('%s training begins'%get_time())
for exp in range(NUMBER_INITIALIZATIONS):
  model = get_network("ConvNet", channel, num_classes, im_size).to(DEVICE)
  model.train()
  net_parameters = list(model.parameters())

  for it in range(ITERATIONS):
    loss = torch.tensor(0.0).to(DEVICE)
    for c in range(num_classes):
      strat_index = c*IMAGE_PER_CLASS
      img_real = images_all[np.random.permutation(class_index[c])[:BATCH_SIZE]]
      lab_real = torch.ones((BATCH_SIZE,), device=DEVICE, dtype=torch.long) * c
      img_syn = syn_image[strat_index: strat_index + IMAGE_PER_CLASS].reshape((IMAGE_PER_CLASS, channel, im_size[0], im_size[1]))
      lab_syn = torch.ones((IMAGE_PER_CLASS,), device=DEVICE, dtype=torch.long) * c
      output_real = model(img_real)
      loss_real = criterion(output_real, lab_real)
      gw_real = torch.autograd.grad(loss_real, net_parameters)
      gw_real = list((_.detach().clone() for _ in gw_real))

      output_syn = model(img_syn)
      loss_syn = criterion(output_syn, lab_syn)
      gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)
      loss += match_mse_loss(gw_syn, gw_real)

    total_loss += loss
    optimizer_img.zero_grad()
    loss.backward()
    optimizer_img.step()

    image_syn_train, label_syn_train = copy.deepcopy(syn_image.detach()), copy.deepcopy(syn_lable.detach())  # avoid any unaware modification
    dst_syn_train = TensorDataset(image_syn_train, label_syn_train)
    train_model(model, image_syn_train, label_syn_train, NUMBER_STEP_OPT)
    
random = "Gaussian noise" if randmon_init else "real training images"
save_name = os.path.join(SAVEPATH, 'vis_%s_%s_%s_%d.png'%("ConvNet", "MHIST", random, IMAGE_PER_CLASS))
image_syn_vis = copy.deepcopy(syn_image.detach().cpu())
save_image(image_syn_vis, save_name, nrow=IMAGE_PER_CLASS)
make_grid(image_syn_vis, nrow=IMAGE_PER_CLASS)
net_eval = get_network("ConvNet", channel, num_classes, im_size).to(DEVICE)
image_syn_eval = copy.deepcopy(syn_image.detach())

print('%s training begins'%get_time())
acc_test = train_and_eval(net_eval, image_syn_eval, syn_lable, dst_test)
print('%s training end'%get_time())